In [1]:
from IPython.display import display
import numpy as np
import bqplot as bq
import ipywidgets as widgets
import random as random
import pandas as pd
import expToLaTeX as elt

In [2]:
## Originally developed June 2018 by Samuel Holen

## Pre-construct model of radioactive decay of a population
## of parent and daughter atoms.
## 

# Constants Related to decay of the parent species to the daughter species
N_parent = 900          # initial number of parent atoms
N_daughter = 0          # initial number of daughter atoms
tau = 1                 # placeholder for the half-life of the parent species 
h = 1.0                 # time step  
mu = np.log(2.) / tau   # constant for decay time distribution 

# Initialize tracking of number of atoms
Parent_counts = []          # list of number of parent atoms 
Dauther_counts = []          # list of number of daughter atoms

# Generate a uniform random distribution of N_parent numbers from 0 to 1
z = np.random.rand(N_parent)

# Function to convert uniform distribution of random numbers to
# a distribution weighted to model radiactive decay. Unsorted representing the
# decay of each object.
decay_times = -np.log(1 - z) / mu
decay_times_sorted = np.sort( decay_times )



# Genereate array of numbers of atoms left
# Adjusted so that each count contains 0 and 900
Parent_counts = np.arange(N_parent,-1, -1, dtype='int')      # Number of parent atoms
Daughter_counts = np.ones_like(Parent_counts)
Daughter_counts = N_parent - Parent_counts   # Number of daughter atoms

# Construct Pandas data fram
# Time column adjusted to include t=0
decay_data = pd.DataFrame()
decay_data['time'] = np.concatenate((np.zeros(1),decay_times_sorted))
decay_data['Parent'] = Parent_counts
decay_data['Daughter'] = Daughter_counts
# Data array for species
species = pd.DataFrame()
species['timeunits'] = ['seconds', 'million years', 'billion years', 'years']
species['parent_long'] = ['Thallium','Uranium','Rubidium','Carbon']
species['daughter_long'] = ['Lead','Thorium','Strontium','Nitrogen']
species['parent_short'] = ['Tl-208','U-235','Rb-87','C-14']
species['daughter_short'] = ['Pb-208','Th-231','Sr-87','N-14']
species['half-lives'] = [3.053 * 60, 703.8, 48.8, 5730]  # seconds, millions of years, millions of years, years



In [3]:
##
## Set up counts versus time plot
## **Default set to Tl-208 for everthing**

# Set up axes
x_time = bq.LinearScale(min = 0, max=max(decay_times*species['half-lives'][0]))
y_number = bq.LinearScale(min = 0, max=N_parent)
y_fraction = bq.LinearScale(min = 0, max=1)

# Forces the number of ticks to be 6 and to start at 0
tick_vals = np.linspace(0, max(decay_times*species['half-lives'][0]),6)

# Labels and scales for Axes
ax_x_time = bq.Axis(label=species['timeunits'][0], scale=x_time,
                    num_ticks=6, tick_values=tick_vals)
ax_y_number = bq.Axis(label='Number of atoms', scale=y_number, orientation='vertical')
ax_y_fraction = bq.Axis(label='Fraction of atoms', scale=y_fraction, orientation='vertical')

# Define tooltip (not working)
def_tt_parent = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2f'], labels=['time', 'Tl-208'])
def_tt_daughter = bq.Tooltip(fields=['x', 'y'], formats=['.2f', '.2f'], labels=['time', 'Pb-208'])
# Define the lines
line_parent = bq.Lines(x=decay_data['time']/species['half-lives'][0], y=[decay_data['Parent'][0]], 
                       scales={'x': x_time, 'y': y_number}, display_legend=True, colors=['red'], 
                       labels=[species['parent_short'][0]], tooltip=def_tt_parent)
line_daughter = bq.Lines(x=decay_data['time']/species['half-lives'][0], y=[decay_data['Daughter'][0]], 
                         scales={'x': x_time, 'y': y_number}, display_legend=True, colors=['blue'], 
                         labels=[species['daughter_short'][0]] ,tooltip=def_tt_daughter)

# Creates figure for plot
fig_counts = bq.Figure(axes=[ax_x_time, ax_y_number], marks=[line_parent, line_daughter], 
                       legend_location='right', legend_style={'fill': 'white'}, 
                       title='Counts versus Time', background_style={'fill': 'black'}, 
                       layout={'width': '500px', 'min_height': '400px'},
                      animation=1000)


In [4]:
# Slider widget to control the figures, controls the amount of time that has passed
Time_slide = widgets.FloatSlider(
    value=0.,
    description='Time',
    min=0.,
    max=max(decay_times*species['half-lives'][0])+1,
    step=h,
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=False,
    readout_format='.1f'
)

# Widget to display the number of parent atoms present
parent_present = widgets.Text(
    value = str(N_parent),
    style = {'description_width': 'initial'},
    description = species['parent_short'][0]+' remaining',
    disabled = True   
)
# Widget to display the number of daughter atoms present
daughter_present = widgets.Text(
    value = str(0),
    style = {'description_width': 'initial'},
    description = species['daughter_short'][0]+' produced',
    disabled = True   
)
# Widgets to label the time slider with units
Time_label = widgets.Label(value=str(Time_slide.value))
unit_label = widgets.Label(value=str(species['timeunits'][0]))

# Checkbox to choose whether to display the number of each species
# or the fraction of each
frac_or_num = widgets.Checkbox(value=False, description='Display as fractions')

# Widget to allow one to choose which species to work with
pick_Species = widgets.RadioButtons(options=species['parent_short'][0:4], 
                                 value='Tl-208', description='Species:', disabled=False,
                                 layout=widgets.Layout(align_content='center', align_items='center', 
                                          display='flex', 
                                          flex_flow='column', height='150px', max_height='200px', 
                                          max_width='300px', min_height='100px', min_width='125px', 
                                          overflow_x='hidden', overflow_y='hidden', width='175px'))
# Scale for population figure
x_sc = bq.LinearScale(min=1, max=30)
y_sc = bq.LinearScale(min=1,max=30)
# Axes for population figure
ax_x = bq.Axis(scale=x_sc, num_ticks=0)
ax_y = bq.Axis(scale=y_sc, orientation='vertical', num_ticks=0)
# Creates an array of x values: [1,2,...,30,1,2,...30,.....,1,2,...,30]
x_ls = []
for i in range(1,31):
    x_ls.append(float(i))
x_ls = x_ls * 30
x_arr = np.array(x_ls)
# Creates an array of y values: [1,1,...,1,2,2,...2,......,30,30,...,30]
y_ls = []
for i in range(1,31):
    y_ls += [float(i)] * 30
y_arr = np.array(y_ls)    
# Creates a color array with the same number of entries as the number of atoms in
# the sample
Colors = ['red'] * N_parent

In [5]:
# Function to update the plots in response to the controllable widgets
def Update(change=None):
    # Sets the half-life of the selected species
    hf = float(species.loc[species.parent_short == pick_Species.value,['half-lives'][0]])
    # Sets the max value of the time slider to the final decay time of the selected species
    Time_slide.max = max(decay_times)*hf+1
    # Changes the color of the correct number of decayed species, randomly distributed
    for i in range(N_parent):
        if Time_slide.value >= decay_times[i]*hf:
            Colors[i] = 'blue'
        else:
            Colors[i] = 'red'
    
    time_arr = hf * decay_data['time']
    # Update the parent and daughter plots
    i = 0
    while i < N_parent + 1 and hf*decay_data['time'][i] < Time_slide.value:        
        i += 1
    if i > 0:
        i -= 1
        
        
    daughter_decay = decay_data['Parent'][0:i+1]
    parent_decay = decay_data['Daughter'][0:i+1] 
    
    # Apply the color change
    population_scat.colors = Colors
    
    line_parent.x = time_arr
    line_daughter.x = time_arr
    
    # Updates the time slider label/units
    if Time_slide.value < 1e3 or pick_Species.value == 'Tl-208':
        unit = str(species.loc[species.parent_short == pick_Species.value,['timeunits'][0]])
        unit_label.value = unit[5:unit.rfind('\n')+1]
        Time_label.value = str(Time_slide.value)  
    elif pick_Species.value == 'U-235' and Time_slide.value >= 1e3:
        unit = str('billion years')
        unit_label.value = unit
        time = elt.exp2LaTeX(Time_slide.value/1000,3)[0]
        Time_label.value = time
    elif pick_Species.value == 'C-14' and Time_slide.value >= 1e3:
        unit = str('thousand years')
        unit_label.value = unit
        time = elt.exp2LaTeX(Time_slide.value/1000,3)[0]
        Time_label.value = time
    else:
        unit = str(species.loc[species.parent_short == pick_Species.value,['timeunits'][0]])
        unit_label.value = unit[5:unit.rfind('\n')+1]    
        Time_label.value = str(round(Time_slide.value))

        
    # Updates the x-axis
    x_time.max = max(decay_times*hf)
    tick_vals = np.linspace(0, max(decay_times*hf),6)
    ax_x_time.tick_values = tick_vals
    ax_x_time.scale = x_time
    ax_x_time.label = unit[5:unit.rfind('\n')+1]
    fig_counts.axes = [ax_x_time,ax_y_number]        
    # Update the units and value displayed on the slider
    if frac_or_num.value == False:
        parent_present.value = str(decay_data['Parent'][i])
        daughter_present.value = str(decay_data['Daughter'][i])

        # Update the x and y arrays for the parent and daughter lines
        line_parent.y = parent_decay
        line_daughter.y = daughter_decay
        line_parent.scales={'x': x_time, 'y': y_number}
        line_daughter.scales={'x': x_time, 'y': y_number}
        fig_counts.marks = [line_parent,line_daughter]

    else:
        # Fraction mode enabled
        fig_counts.axes = [ax_x_time,ax_y_fraction]
        # Update the x and y arrays for the parent and daughter lines
        line_parent.y = (1/900)*parent_decay
        line_daughter.y = (1/900)*daughter_decay
        line_parent.scales={'x': x_time, 'y': y_fraction}
        line_daughter.scales={'x': x_time, 'y': y_fraction}
        fig_counts.marks = [line_parent,line_daughter]
        parent_present.value = '{:.2f}'.format((1/900)*decay_data['Parent'][i])
        daughter_present.value = '{:.2f}'.format((1/900)*decay_data['Daughter'][i])
    
    # Update the legend
    parent_label = str(species.loc[species.parent_short == pick_Species.value,['parent_short'][0]])
    daughter_label = str(species.loc[species.parent_short == pick_Species.value,['daughter_short'][0]])
    line_parent.labels = [parent_label[5:parent_label.find('\n')+1]]
    line_daughter.labels = [daughter_label[5:daughter_label.find('\n')+1]]
    # Update the species in the box that shows how many are present    
    parent_present.description = pick_Species.value+' produced'
    daughter_present.description = daughter_label[5:daughter_label.find('\n')+1]+' remaining'
    

    




In [6]:
# Plot the population model
population_scat = bq.Scatter(x=x_arr, y=y_arr, scales={'x': x_sc, 'y': y_sc}, colors =['red'])
# Update the values/colors
Time_slide.observe(Update, names=['value'])
parent_present.observe(Update, names=['value'])
daughter_present.observe(Update, names=['value'])
pick_Species.observe(Update, names=['value'])
Time_label.observe(Update, names=['value'])
frac_or_num.observe(Update, names=['value'])

# Figure for the population
fig = bq.Figure(title='Population', marks=[population_scat], axes=[ax_x, ax_y], 
                background_style={'fill' : 'black'},padding_x = 0.025,
               min_aspect_ratio=1, max_aspect_ratio=1)
# Boxes to organize display
slide_box = widgets.HBox([Time_slide, Time_label,unit_label])
right_box = widgets.VBox([fig, slide_box, frac_or_num])
left_box = widgets.VBox([fig_counts, widgets.HBox([widgets.VBox([parent_present,daughter_present]), pick_Species])])
right_box.layout.width = '50%'
left_box.layout.width = '50%'
# Final display
Final = widgets.HBox([left_box,right_box])
Final.layout.overflow_x = 'hidden'

display(Final)